# Web Scrapping

## Introduction

<b><i>Data source:</b></i>
https://boardgamegeek.com/browse/boardgame  <b>OR</b> <br>
--> https://boardgamegeek.com/browse/boardgame/page/1

(An inner page looks like: https://boardgamegeek.com/boardgame/224517/brass-birmingham)

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from random import randint
from time import sleep #allows to input a delay between 'clicking' from one page to another so our behavior looks like human

## Connection to my 'dummy' account

To avoid the connecting issue (otherwise we get limited to 1000 rows maximum), we use a method proposed here to keep our cookies and headers from our connection. After a few days, the info might naturally change so one should do the method again.

<i><b>Link:</i></b> https://stackoverflow.com/questions/23102833/how-to-scrape-a-website-which-requires-login-using-python-and-beautifulsoup/23103784#23103784
<br>
--> Apply the second solution with cURL and https://curlconverter.com/

<i><b>PS</b></i>: There is maybe an alternative way to get to the same thing, but we won't get there  <br>
https://stackoverflow.com/questions/13925983/login-to-website-using-urllib2-python-2-7

In [43]:
cookies = {
    'FCCDCF': '%5Bnull%2Cnull%2Cnull%2C%5B%22CP9Q9kAP9Q9kAEsACBFRAwEgAAAAAEPgACiQAAAOhQD2F2K2kKFkPCmQWYAQBCijYEAhQAAAAkCBIAAgAUgQAgFIIAgAIFAAAAAAAAAQEgCQAAQABAAAIACgAAAAAAIAAAAAAAQQAAAAAIAAAAAAAAEAAAAAAAQAAAAIAABEgCAAQQAEAAAAAAAQAAAAAAAAAAABAAA%22%2C%222~~dv.2072.70.89.93.108.122.149.196.2253.2299.259.2357.311.313.323.2373.338.358.2415.415.449.2506.2526.486.494.495.2568.2571.2575.540.574.2624.609.2677.864.981.1029.1048.1051.1095.1097.1126.1205.1211.1276.1301.1365.1415.1423.1449.1570.1577.1598.1651.1716.1735.1753.1765.1870.1878.1889.1896.1958%22%2C%22A560738A-122E-4B51-B968-7994D6C9E38E%22%5D%5D',
    'cc_cookie': '%7B%22level%22%3A%5B%22necessary%22%5D%2C%22revision%22%3A1%2C%22data%22%3Anull%2C%22rfc_cookie%22%3Atrue%7D',
    'bggusername': 'Irontest',
    'bggpassword': 'nhyy68l5vny1srd10rduqbrjcobsm1udc',
    'SessionID': '1a8759180f8f796082abe96595a4a0c1a1f8d053u3859721',
    '__eoi': 'ID=0d10bef624742d14:T=1713449399:RT=1713452023:S=AA-Afjb8azZ-uxQU98rEZZtzelNW',
}

headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'fr-FR,fr;q=0.9',
    'cache-control': 'max-age=0',
    # 'cookie': 'FCCDCF=%5Bnull%2Cnull%2Cnull%2C%5B%22CP9Q9kAP9Q9kAEsACBFRAwEgAAAAAEPgACiQAAAOhQD2F2K2kKFkPCmQWYAQBCijYEAhQAAAAkCBIAAgAUgQAgFIIAgAIFAAAAAAAAAQEgCQAAQABAAAIACgAAAAAAIAAAAAAAQQAAAAAIAAAAAAAAEAAAAAAAQAAAAIAABEgCAAQQAEAAAAAAAQAAAAAAAAAAABAAA%22%2C%222~~dv.2072.70.89.93.108.122.149.196.2253.2299.259.2357.311.313.323.2373.338.358.2415.415.449.2506.2526.486.494.495.2568.2571.2575.540.574.2624.609.2677.864.981.1029.1048.1051.1095.1097.1126.1205.1211.1276.1301.1365.1415.1423.1449.1570.1577.1598.1651.1716.1735.1753.1765.1870.1878.1889.1896.1958%22%2C%22A560738A-122E-4B51-B968-7994D6C9E38E%22%5D%5D; cc_cookie=%7B%22level%22%3A%5B%22necessary%22%5D%2C%22revision%22%3A1%2C%22data%22%3Anull%2C%22rfc_cookie%22%3Atrue%7D; bggusername=Irontest; bggpassword=nhyy68l5vny1srd10rduqbrjcobsm1udc; SessionID=1a8759180f8f796082abe96595a4a0c1a1f8d053u3859721; __eoi=ID=0d10bef624742d14:T=1713449399:RT=1713452023:S=AA-Afjb8azZ-uxQU98rEZZtzelNW',
    'dnt': '1',
    'referer': 'https://boardgamegeek.com/browse/boardgame/page/2',
    'sec-ch-ua': '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
}

response = requests.get('https://boardgamegeek.com/browse/boardgame/page/1', cookies=cookies, headers=headers)

## Scraping the data

In [57]:
# Function to scrape data from a single page
def scrape_page(url):
    response = requests.get(url, cookies=cookies, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr', {'id': lambda x: x and x.startswith('row_')})
    
    data = []
    for row in rows:
        cells = row.find_all('td')
        rank = cells[0].text.strip()
        title_cell = cells[2]
        title_link = title_cell.find('a', class_='primary')
        title = title_link.text.strip() if title_link else 'N/A'
        game_url = title_link['href'] if title_link else 'N/A'
        full_game_url = f'https://boardgamegeek.com{game_url}' if game_url != 'N/A' else 'N/A'
        geek_rating = cells[4].text.strip() if cells[4].text.strip() else 'N/A'
        avg_rating = cells[5].text.strip() if cells[5].text.strip() else 'N/A'
        num_voters = cells[6].text.strip() if cells[6].text.strip() else 'N/A'
        data.append([rank, title, full_game_url, geek_rating, avg_rating, num_voters])
    
    sleep(randint(3,10))
    
    return data


In [58]:
# Scrape all pages and concatenate data into a single list
all_data = []
base_url = 'https://boardgamegeek.com/browse/boardgame/page/'
num_pages = 22

for page_num in range(1, num_pages + 1):
    page_url = f'{base_url}{page_num}'
    page_data = scrape_page(page_url)
    all_data.extend(page_data)

# Create DataFrame
df = pd.DataFrame(all_data, columns=['Board Game Rank', 'Title', 'URL', 'Geek Rating', 'Avg Rating', 'Num Voters'])

In [59]:
# Print the DataFrame
display(df)

,Board Game Rank,Title,URL,Geek Rating,Avg Rating,Num Voters
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.417,8.60,44645
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.382,8.53,53072
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.358,8.59,61672
3,4,Ark Nova,https://boardgamegeek.com/boardgame/342942/ark...,8.333,8.54,41833
4,5,Twilight Imperium: Fourth Edition,https://boardgamegeek.com/boardgame/233078/twi...,8.243,8.61,23340
...,...,...,...,...,...,...
2195,2196,Subbuteo,https://boardgamegeek.com/boardgame/3720/subbuteo,6.152,6.99,1840
2196,2197,Agents of SMERSH,https://boardgamegeek.com/boardgame/111105/age...,6.152,6.96,1679
2197,2198,The Princes of Machu Picchu,https://boardgamegeek.com/boardgame/36811/prin...,6.152,6.95,1708
2198,2199,Take it Easy!,https://boardgamegeek.com/boardgame/128/take-i...,6.151,6.61,3202
